In [1]:
%load_ext autoreload
%autoreload 2

In [40]:
import pandas as pd 
import numpy as np 
from gptchem.plotsettings import *

from glob import glob 
from fastcore.xtras import load_pickle

from gptchem.evaluator import is_in_pubchem, is_valid, get_sa_scores
from collections import defaultdict

In [41]:
all_res = glob("out/*/*.pkl")

In [42]:
len(all_res)

3

In [43]:
compiled_res = []
generated_smiles = set()
generated_smiles_at_temp = defaultdict(set)

for res_path in all_res:
    res = load_pickle(res_path)
    for temp_res in res["res_at_temp"]:
        try:
            generated_smiles = [s.split()[0] for s in temp_res["generated_smiles"]]
            
            metrics = {
                "noise_level": res["noise_level"],
                "temp": temp_res["temperature"],
                "frac_valid": temp_res["frac_valid"],
                "frac_unique": temp_res["frac_unique"],
                "frac_smiles_in_train": temp_res["frac_smiles_in_train"],
                "frac_smiles_in_pubchem": sum([is_in_pubchem(s.split()[0]) for s in temp_res["valid_smiles"]]) / len(temp_res["valid_smiles"]), 
                "kld": temp_res["kld"],
                "frechet_d": temp_res["frechet_d"],
                "frechet_score": temp_res["frechet_score"],
                "train_size": res["train_size"],

            }
            valid_smiles = [s for s in generated_smiles if is_valid(s)]


            metrics['sa_score'] = np.mean(get_sa_scores(valid_smiles))


            v = [s for s in generated_smiles if is_valid(s)]
            generated_smiles_at_temp[temp_res["temperature"]].update(v)

          
            for me, v in temp_res['constrain_satisfaction'].items():
                metrics[me] = v

            
            compiled_res.append(metrics)

        except Exception as e:
            print(e)
            pass

'PUGREST.BadRequest: error: '
'PUGREST.BadRequest: error: '
'PUGREST.BadRequest: error: '


In [46]:
compiled_res = pd.DataFrame(compiled_res)

In [47]:
compiled_res.groupby(['train_size', 'temp', 'noise_level']).mean()

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_46964/3231958506.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  compiled_res.groupby(['train_size', 'temp', 'noise_level']).mean()


frac_valid  frac_unique  frac_smiles_in_train  \
train_size temp noise_level                                                  
100        0.00 0.01               0.01     1.000000                   0.0   
           0.10 0.01               0.11     0.363636                   0.0   
           0.20 0.01               0.14     0.428571                   0.0   
           0.50 0.01               0.13     1.000000                   0.0   
           0.75 0.01               0.08     1.000000                   0.0   
           1.00 0.01               0.03     1.000000                   0.0   
           1.25 0.01               0.02     1.000000                   0.0   
           1.50 0.01               0.10     1.000000                   0.0   
300        0.00 0.01               1.00     0.050000                   0.0   
           0.10 0.01               0.95     0.094737                   0.0   
           0.20 0.01               0.80     0.325000                   0.0   
           0.50 0.01               0.47     0.978723                   0.0   
           0.75 0.01               0.13     1.000000                   0.0   
           1.00 0.01               0.04     1.000000                   0.0   
           1.25 0.01               0.04     1.000000                   0.0   
           1.50 0.01               0.05     1.000000                   0.0   
1000       0.00 0.01               0.02     0.500000                   0.0   
           0.10 0.01               0.13     0.538462                   0.0   
           0.20 0.01               0.40     0.800000                   0.0   
           0.50 0.01               0.54     1.000000                   0.0   
           0.75 0.01               0.42     1.000000                   0.0   
           1.00 0.01               0.18     1.000000                   0.0   
           1.25 0.01               0.04     1.000000                   0.0   
           1.50 0.01               0.10     1.000000                   0.0   

                             frac_smiles_in_pubchem       kld   frechet_d  \
train_size temp noise_level                                                 
100        0.00 0.01                       1.000000       NaN         NaN   
           0.10 0.01                       1.000000       NaN   71.626816   
           0.20 0.01                       1.000000       NaN   78.115788   
           0.50 0.01                       0.692308  0.201572   57.633018   
           0.75 0.01                       0.500000  0.317779   56.856478   
           1.00 0.01                       0.333333  0.367588   63.112198   
           1.25 0.01                       1.000000       NaN   58.135285   
           1.50 0.01                       1.000000       NaN   61.769426   
300        0.00 0.01                       0.880000       NaN   67.562637   
           0.10 0.01                       0.884211  0.273844   64.706283   
           0.20 0.01                       0.850000  0.177438   53.685822   
           0.50 0.01                       0.659574  0.162301   43.812644   
           0.75 0.01                       0.461538  0.339192   57.747864   
           1.00 0.01                       0.500000       NaN   58.824309   
           1.25 0.01                       0.750000  0.210938   50.495081   
           1.50 0.01                       1.000000       NaN   53.658447   
1000       0.00 0.01                       0.000000       NaN         NaN   
           0.10 0.01                       0.692308  0.128015  104.440885   
           0.20 0.01                       0.700000  0.184957   60.521833   
           0.50 0.01                       0.333333  0.250655   39.347382   
           0.75 0.01                       0.285714  0.314459   41.959614   
           1.00 0.01                       0.055556  0.411990   53.991955   
           1.25 0.01                       0.250000  0.275521  100.661357   
           1.50 0.01                       0.900000       NaN   68.077822  